# Computing stats per user

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from mbti_type_from_text.db_utils import create_connection
import pandas as pd
import plotly.express as px
import re
import numpy as np
import emot  # https://github.com/NeelShah18/emot
import nltk

In [2]:
db_connection = create_connection("../data/reddit.db")

# 1. Grouping messages by user

In [3]:
comments_df = pd.read_sql(sql="SELECT * FROM Comments", con=db_connection)
# output of reddit_exploration 1. (users with mbti_type)
users_df = pd.read_feather("../data/users_df_with_mbti_type.feather")\
    .fillna(np.nan)  # apparently, feather treats "NaN" as "None", converting back to "NaN"

In [4]:
users_df.head()

,id,name,mbti_type
0,6i0rnp1p,igid221,INFJ
1,1s8dnq6p,Sheilaahmad,NaN
2,b708k,lzkbloodmage,INFJ
3,mkfu3,ShannyPantsxo,NaN
4,vh9kmmx,lala2love,NaN


In [5]:
# In this case, we want empty titles to not count as one:
# replaces field that's entirely space (or empty) with NaN
comments_df["title"] = comments_df.title\
    .replace(r'^\s*$', np.nan, regex=True)

content_per_user = comments_df\
    .groupby("user_id").agg({"title": list, "content": list})\
    .rename(columns={"title": "titles", "content": "contents"})
# ignores any nan title or content
for c in ["titles", "contents"]:
    content_per_user[c] = content_per_user[c].apply(lambda l: [e for e in l if not pd.isna(e)])
    
content_per_user["all_content"] = content_per_user\
    .apply(lambda row: row["titles"] + row["contents"], axis=1)\

# a message is a title or a content, treated equally
content_per_user["n_msgs"] = content_per_user.all_content.apply(len)

content_per_user["msgs"] = content_per_user.all_content\
    .apply(lambda l: " ".join(l))

content_per_user["character_count"] = content_per_user.msgs.apply(len)
content_per_user.head()

,titles,contents,all_content,n_msgs,msgs,character_count
user_id,,,,,,
100fu2,[],[ENFJ 4w3 (28M) here who dated an ENFP 7w6 (23...,[ENFJ 4w3 (28M) here who dated an ENFP 7w6 (23...,1,ENFJ 4w3 (28M) here who dated an ENFP 7w6 (23F...,965
100sih,[],[Path of least resistance so I can go back to ...,[Path of least resistance so I can go back to ...,3,Path of least resistance so I can go back to w...,485
100wx2,[Ow],[Hurt my leg and now bedridden. Life is full o...,"[Ow, Hurt my leg and now bedridden. Life is fu...",3,Ow Hurt my leg and now bedridden. Life is full...,103
101tuq,[],[I had similar problems when I was typing myse...,[I had similar problems when I was typing myse...,5,I had similar problems when I was typing mysel...,2091
104bxl,[],[as a teenager I searched for some romantic an...,[as a teenager I searched for some romantic an...,1,as a teenager I searched for some romantic and...,331


In [6]:
content_per_user = content_per_user\
    .merge(users_df[["id", "mbti_type"]].rename(columns={"id": "user_id"}), how="left", left_index=True, right_on="user_id")
content_per_user.head()

,titles,contents,all_content,n_msgs,msgs,character_count,user_id,mbti_type
3600,[],[ENFJ 4w3 (28M) here who dated an ENFP 7w6 (23...,[ENFJ 4w3 (28M) here who dated an ENFP 7w6 (23...,1,ENFJ 4w3 (28M) here who dated an ENFP 7w6 (23F...,965,100fu2,NaN
4118,[],[Path of least resistance so I can go back to ...,[Path of least resistance so I can go back to ...,3,Path of least resistance so I can go back to w...,485,100sih,ENTP
3958,[Ow],[Hurt my leg and now bedridden. Life is full o...,"[Ow, Hurt my leg and now bedridden. Life is fu...",3,Ow Hurt my leg and now bedridden. Life is full...,103,100wx2,NaN
2870,[],[I had similar problems when I was typing myse...,[I had similar problems when I was typing myse...,5,I had similar problems when I was typing mysel...,2091,101tuq,INTJ
1556,[],[as a teenager I searched for some romantic an...,[as a teenager I searched for some romantic an...,1,as a teenager I searched for some romantic and...,331,104bxl,NaN


In [7]:
content_per_user = content_per_user[["user_id", "mbti_type", "n_msgs", "character_count", "msgs"]]

# 2. Counting and removing items

## What should be cleaned?
### Counting and removing
* [x] \n (sometimes not spaced out from the rest) -> count and remove
* [x] emojis? 😂 -> Count and remove
* [x] emoticons :) -> Count and remove
* [x] urls -> count and remove
* [x] numbers -> count and remove
* markdown tags -> count each and remove

### Cleaning
* remove special chars as a catch all
* tokenize
* remove stopwords
* lematize

For each item, in a specific order:
1. count (create a new col)
2. remove

Seems like a good way to avoid conflicts, such as an emoticon found in a url

In [8]:
content_per_user["msgs_raw"] = content_per_user.msgs  # keeping a copy of the original msgs

## 2.A. Counting and removing items

In [9]:
regexes = {
    "url": r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)",
    "number_group": r"([0-9]+)",  # also matches on abc1de, do we want that?
    "line_break": r"(\n)",
    "punctuation": r"[.!?\\-]"
}

In [10]:
# Handling emojis/emoticons with https://github.com/NeelShah18/emot

def count_emojis_and_emoticons(text):
    # Note: emoticons count is partial: ^^ is not recognized for example
    _sum = 0
    for match_obj in [emot.emoji(text), emot.emoticons(text)]:
        if type(match_obj) is not list:  # apparently, when there are no matches, output is list
            _sum += len(match_obj["value"])
    return _sum

def remove_emojis(text):
    match_obj = emot.emoji(text)
    if type(match_obj) is list:
        return text
    
    unique_matches = set(match_obj["value"])
    for match in unique_matches:
        text = text.replace(match, '')
    return text

def remove_emoticons(text):
    match_obj = emot.emoticons(text)
    if type(match_obj) is list:
        return text
    
    unique_matches = set(match_obj["value"])
    for match in unique_matches:
        text = text.replace(match, '')
    return text

def handle_emojis_and_emoticons(df, msgs_col_name: str):
    df["emoji_and_emoticon_count"] = df[msgs_col_name].apply(count_emojis_and_emoticons)
    df[msgs_col_name] = df[msgs_col_name]\
        .apply(remove_emoticons)\
        .apply(remove_emojis)  # maybe we could keep emojis as tokens for vectorization?

In [11]:
def count_and_remove(df, msgs_col_name: str, regex_names: [str]):
    """
    For each regex, counts occurences from the specified col into a new column, and removes them
    """
    for regex_name, regex in regexes.items():
        if regex_name in regex_names:
            df[f"{regex_name}_count"] = df[msgs_col_name]\
                .apply(lambda m: len(re.findall(regex, m)))  # counts
            df[msgs_col_name] = df[msgs_col_name].apply(lambda m: re.sub(regex, '', m)) # removes
            
def add_means_per_msg(df, msgs_count_col: str, char_count_col: str, drop_count_cols: bool):
    for count_col in [c for c in df.columns if c.endswith("_count")]:
        mean_col_name = "{item}_msg_mean".format(item=count_col[:-6])
        df[mean_col_name] = df.apply(lambda row: row[count_col] / row[msgs_count_col], axis=1)
        if count_col != "character_count":  # mean char per char is not super useful 
            mean_col_name = "{item}_char_mean".format(item=count_col[:-6])
            df[mean_col_name] = df.apply(lambda row: row[count_col] / row[char_count_col], axis=1)
    if drop_count_cols:
        df = df[[c for c in df.columns if not c.endswith("_count") or c == "character_count"]]
    return df
            
def clean_and_count_all_items(df, msgs_col_name: str):
    """
    For each regex, counts occurences from the specified col into a new column, and removes them
    Some are done before remove emojis/emoticons, some after
    Can be applied to any df having a column containing msgs as string
    """
    to_do_before_emojis_and_emoticons = ["url"]
    
    count_and_remove(df, msgs_col_name, to_do_before_emojis_and_emoticons)
    
    handle_emojis_and_emoticons(df, msgs_col_name)
    
    to_do_after_emojis_and_emoticons = [r for r in list(regexes.keys()) if r not in to_do_before_emojis_and_emoticons]
    count_and_remove(df, msgs_col_name, to_do_after_emojis_and_emoticons)
    return df

In [12]:
%%time
content_per_user = clean_and_count_all_items(content_per_user, "msgs")

Wall time: 15.3 s


In [13]:
content_per_user.head()

,user_id,mbti_type,n_msgs,character_count,msgs,msgs_raw,url_count,emoji_and_emoticon_count,number_group_count,line_break_count,punctuation_count
3600,100fu2,NaN,1,965,ENFJ w (M) here who dated an ENFP w (F)Basical...,ENFJ 4w3 (28M) here who dated an ENFP 7w6 (23F...,0,1,9,11,11
4118,100sih,ENTP,3,485,Path of least resistance so I can go back to w...,Path of least resistance so I can go back to w...,0,0,0,0,7
3958,100wx2,NaN,3,103,Ow Hurt my leg and now bedridden Life is full ...,Ow Hurt my leg and now bedridden. Life is full...,0,0,0,0,5
2870,101tuq,INTJ,5,2091,I had similar problems when I was typing mysel...,I had similar problems when I was typing mysel...,0,0,3,10,28
1556,104bxl,NaN,1,331,as a teenager I searched for some romantic and...,as a teenager I searched for some romantic and...,0,0,0,4,3


In [14]:
content_per_user = add_means_per_msg(df=content_per_user, msgs_count_col="n_msgs",
                                     char_count_col="character_count", drop_count_cols=True)
content_per_user.head()

,user_id,mbti_type,n_msgs,character_count,msgs,msgs_raw,character_msg_mean,url_msg_mean,url_char_mean,emoji_and_emoticon_msg_mean,emoji_and_emoticon_char_mean,number_group_msg_mean,number_group_char_mean,line_break_msg_mean,line_break_char_mean,punctuation_msg_mean,punctuation_char_mean
3600,100fu2,NaN,1,965,ENFJ w (M) here who dated an ENFP w (F)Basical...,ENFJ 4w3 (28M) here who dated an ENFP 7w6 (23F...,965.000000,0.0,0.0,1.0,0.001036,9.0,0.009326,11.0,0.011399,11.000000,0.011399
4118,100sih,ENTP,3,485,Path of least resistance so I can go back to w...,Path of least resistance so I can go back to w...,161.666667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,2.333333,0.014433
3958,100wx2,NaN,3,103,Ow Hurt my leg and now bedridden Life is full ...,Ow Hurt my leg and now bedridden. Life is full...,34.333333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.666667,0.048544
2870,101tuq,INTJ,5,2091,I had similar problems when I was typing mysel...,I had similar problems when I was typing mysel...,418.200000,0.0,0.0,0.0,0.000000,0.6,0.001435,2.0,0.004782,5.600000,0.013391
1556,104bxl,NaN,1,331,as a teenager I searched for some romantic and...,as a teenager I searched for some romantic and...,331.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,4.0,0.012085,3.000000,0.009063


## 2.B Cleaning msgs into tokens

In [15]:
from nltk.tokenize import word_tokenize

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W|_', ' ', text) # removes special chars
    text = re.sub(r'\s+', ' ', text) # removes multiple spaces
    text = re.sub(r'^\s|\s$', '', text) # removes space at the start or end of the string
    
    tokens = word_tokenize(text)  # tokenizes
    
    stopwords = nltk.corpus.stopwords.words("english")  # are all msgs in english?
    tokens = [token for token in tokens if token not in stopwords] # removes stopwords
    tokens = [token for token in tokens if len(token) > 1]  # removes 1-char tokens
    
    wn = nltk.WordNetLemmatizer()
    tokens = [wn.lemmatize(token) for token in tokens] # lematizes=roots words
    return tokens

example_msg = content_per_user.iloc[0].msgs
print("{} \n     |\n     v\n{}".format(
    example_msg,
    clean_text(example_msg)
))

ENFJ w (M) here who dated an ENFP w (F)Basically, when something goes unexpectedly wrong, it is like the end of the world to me Nothing can go right anymore and it takes time for me to get back on track But what I love most is getting some air from time to time A few months back I was quite exhausted of being with her because I had to focus on my things first and on the other hand she was constantly on demand of new experiences and so on, even though she is/was literally the perfect girl to meAnd actually she ended up dumping me and I was so devastated for the next  monthsSo, IMO the best move you can do to get him back is to trully ignore him and enjoy being with others and friends He will definitely notice you are doing great with others while he still hermitting and overthinkingWe love to be loved but we hate more to be ignored, bear this in mindI can assure you within  weeks you'll be dating again ^^' (probably  ) 
     |
     v
['enfj', 'dated', 'enfp', 'basically', 'something', '

In [16]:
%%time
content_per_user["tokens"] = content_per_user.msgs.apply(clean_text)
content_per_user[["msgs", "tokens"]].head()

Wall time: 8.51 s


,msgs,tokens
3600,ENFJ w (M) here who dated an ENFP w (F)Basical...,"[enfj, dated, enfp, basically, something, go, ..."
4118,Path of least resistance so I can go back to w...,"[path, least, resistance, go, back, whatever, ..."
3958,Ow Hurt my leg and now bedridden Life is full ...,"[ow, hurt, leg, bedridden, life, full, well, p..."
2870,I had similar problems when I was typing mysel...,"[similar, problem, typing, intj, hard, took, y..."
1556,as a teenager I searched for some romantic and...,"[teenager, searched, romantic, hidden, spiritu..."


In [17]:
content_per_user_output = content_per_user[["user_id", "mbti_type", "n_msgs", "character_count", "tokens"] + [c for c in content_per_user.columns if c.endswith("_mean")]]\
    .reset_index(drop=True)
content_per_user_output.to_feather("../data/stats_and_tokens_per_user.feather")

In [18]:
content_per_user_output.head()

,user_id,mbti_type,n_msgs,character_count,tokens,character_msg_mean,url_msg_mean,url_char_mean,emoji_and_emoticon_msg_mean,emoji_and_emoticon_char_mean,number_group_msg_mean,number_group_char_mean,line_break_msg_mean,line_break_char_mean,punctuation_msg_mean,punctuation_char_mean
0,100fu2,NaN,1,965,"[enfj, dated, enfp, basically, something, go, ...",965.000000,0.0,0.0,1.0,0.001036,9.0,0.009326,11.0,0.011399,11.000000,0.011399
1,100sih,ENTP,3,485,"[path, least, resistance, go, back, whatever, ...",161.666667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,2.333333,0.014433
2,100wx2,NaN,3,103,"[ow, hurt, leg, bedridden, life, full, well, p...",34.333333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.666667,0.048544
3,101tuq,INTJ,5,2091,"[similar, problem, typing, intj, hard, took, y...",418.200000,0.0,0.0,0.0,0.000000,0.6,0.001435,2.0,0.004782,5.600000,0.013391
4,104bxl,NaN,1,331,"[teenager, searched, romantic, hidden, spiritu...",331.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,4.0,0.012085,3.000000,0.009063


In [19]:
def aggregate_from_individuals(stats_and_tokens_per_user, by: [str]):
    """
    Groups individuals by the specified column(s)
    """
    df = stats_and_tokens_per_user
    aggregation = {
        **{"user_id": len,
           "n_msgs": [sum, "mean"],
           "character_count": sum,
           "tokens": sum},
        **{mean_c: np.mean for mean_c in [c for c in df.columns if c.endswith("_mean")]} # add std?
    }
    
    df_per_type = df.groupby(by, dropna=False)\
        .agg(aggregation)
    
    # flattens the multi_index
    df_per_type.columns = ["_".join(x) for x in df_per_type.columns.ravel()]
    df_per_type.columns = [c.replace('_mean_mean', '_mean').replace('_count_count', '_count') for c in df_per_type.columns]
    df_per_type = df_per_type.rename(
        columns={
            "tokens_sum": "tokens",
            "character_count_sum": "characters",
            "user_id_len": "individuals",
            "n_msgs_sum": "msgs",
            "n_msgs_mean": "msgs_per_individual"
        }
    )
    return df_per_type

df_per_mbti_type = aggregate_from_individuals(content_per_user_output, by=["mbti_type"])
df_per_mbti_type.reset_index(drop=True).to_feather("../data/stats_and_tokens_per_mbti_type.feather")
df_per_mbti_type

<ipython-input-19-e6b0a3b9c2aa>:18: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df_per_type.columns = ["_".join(x) for x in df_per_type.columns.ravel()]


,individuals,msgs,msgs_per_individual,characters,tokens,character_msg_mean,url_msg_mean,url_char_mean,emoji_and_emoticon_msg_mean,emoji_and_emoticon_char_mean,number_group_msg_mean,number_group_char_mean,line_break_msg_mean,line_break_char_mean,punctuation_msg_mean,punctuation_char_mean
mbti_type,,,,,,,,,,,,,,,,
ENFJ,105,396,3.771429,149528,"[yeah, felt, although, understand, ability, ma...",377.943008,0.020785,0.000077,0.172415,0.002015,0.366218,0.003170,1.802482,0.003463,5.439475,0.020784
ENFP,171,621,3.631579,171011,"[wan, na, join, gang, hsp, happy, meet, oh, ac...",276.627288,0.017149,0.000116,0.179677,0.004745,0.293508,0.003751,1.254692,0.002837,3.857329,0.015681
ENTJ,142,738,5.197183,166606,"[obsession, selfimprovement, spent, teenage, y...",219.381910,0.016341,0.000106,0.081791,0.000906,0.276968,0.001552,1.005389,0.003852,3.029230,0.017193
ENTP,141,598,4.241135,184050,"[path, least, resistance, go, back, whatever, ...",293.342750,0.014554,0.000090,0.112216,0.000996,0.333732,0.001163,1.256943,0.003409,4.011158,0.017461
ENXP,1,8,8.000000,7891,"[estjs, would, change, could, give, read, seem...",986.375000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000253,3.000000,0.003041,12.750000,0.012926
ESFJ,31,201,6.483871,50569,"[vocal, inflection, yes, people, readily, unde...",218.322544,0.040323,0.000580,0.025058,0.000256,0.134095,0.000864,0.726921,0.003180,3.400043,0.015921
ESFP,35,229,6.542857,58497,"[yeah, grad, student, doctoral, program, none,...",238.223281,0.042857,0.000305,0.179522,0.001589,0.269883,0.001855,0.687075,0.002140,3.191472,0.019014
ESTJ,36,194,5.388889,84358,"[cognitive, function, opposite, order, make, s...",459.471908,0.044491,0.000378,0.027778,0.000091,0.273267,0.000730,2.092926,0.003875,5.693913,0.011576
ESTP,53,263,4.962264,85611,"[oh, yeah, relate, need, develop, like, even, ...",260.665533,0.020357,0.000132,0.052550,0.000602,0.253586,0.001238,1.142247,0.003264,3.149005,0.013267
